#Install libraries and CLI

In [3]:
!pip install -q google-adk google-genai python-dotenv


#Configure Gemini and sanity check

In [4]:
import os
from google import genai

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

client = genai.Client(api_key=GEMINI_API_KEY)

resp = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="Say hello as a command-line analysis tool.",
)
print(resp.text)


```
Analyzing...

OK.

Output:

Hello.
```



#Implement CLI script

In [5]:
%%writefile gemini_cli_tool.py
import sys
import os
from google import genai

def main():
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        print("ERROR: GEMINI_API_KEY is not set.", file=sys.stderr)
        sys.exit(1)

    client = genai.Client(api_key=api_key)

    # If arguments are given, use them; otherwise read stdin
    if len(sys.argv) > 1:
        text = " ".join(sys.argv[1:])
    else:
        text = sys.stdin.read()

    if not text.strip():
        print("ERROR: No input text provided.", file=sys.stderr)
        sys.exit(1)

    resp = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=text,
    )
    print(resp.text.strip())

if __name__ == "__main__":
    main()


Writing gemini_cli_tool.py


#Test the CLI script

In [6]:
!python gemini_cli_tool.py "Explain in one sentence what this CLI tool does."


This CLI tool likely automates tasks, manipulates data, or provides information based on command-line input, without a graphical user interface.


#Wrapper function and FunctionTool

In [7]:
import subprocess
from google.adk.tools import FunctionTool

def run_gemini_cli(command: str) -> str:
    """
    Runs our gemini_cli_tool.py with the given command text.
    Returns the CLI output as a string.
    """
    proc = subprocess.Popen(
        ["python", "gemini_cli_tool.py", command],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    out, err = proc.communicate()

    if err.strip():
        return f"[CLI error]\n{err.strip()}\n\n[Output]\n{out.strip()}"
    return out.strip()

gemini_cli_adk_tool = FunctionTool(func=run_gemini_cli)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [8]:
print(run_gemini_cli("Summarize what a B2B SaaS CRM is in one sentence."))


A B2B SaaS CRM is a cloud-based software solution designed to help businesses manage and optimize their relationships with other businesses throughout the customer lifecycle, from lead generation to post-sale support.


#CLI agent definition

In [9]:
from google.adk.agents import LlmAgent

cli_agent = LlmAgent(
    name="cli_tool_agent",
    model="gemini-2.0-flash-001",
    instruction=(
        "You are an assistant that can call an external Gemini CLI tool via the "
        "run_gemini_cli function. "
        "When the user asks for deeper analysis or a rich summary, "
        "use the tool instead of answering directly, then summarize the tool's output."
    ),
    tools=[gemini_cli_adk_tool],
)


#Runner and session

In [10]:
from google.adk.runners import InMemoryRunner

APP_NAME = "cli_agent_app"
USER_ID = "demo-cli-user"

runner = InMemoryRunner(agent=cli_agent, app_name=APP_NAME)

# Colab lets us use top-level await
session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
)

print("Session ID:", session.id)


Session ID: 1717628b-311f-4c93-9c99-e851e4067d3d


#Helper to query CLI agent

In [11]:
from google.genai.types import Content, Part

def run_cli_agent(prompt: str):
    user_msg = Content(
        role="user",
        parts=[Part.from_text(text=prompt)],
    )

    print("=== USER PROMPT ===")
    print(prompt)
    print("\n=== AGENT RESPONSE ===")

    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=user_msg,
    ):
        if hasattr(event, "is_final_response") and event.is_final_response():
            if event.content:
                for part in event.content.parts:
                    text = getattr(part, "text", None)
                    if text:
                        print(text.strip())

# try it once
run_cli_agent(
    "Using the external CLI tool, give me a detailed, structured explanation of B2B SaaS CRM for mid-market companies."
)


=== USER PROMPT ===
Using the external CLI tool, give me a detailed, structured explanation of B2B SaaS CRM for mid-market companies.

=== AGENT RESPONSE ===


Okay! Here's a breakdown of B2B SaaS CRM for mid-market companies, based on the output from the tool:

**B2B SaaS CRM Defined:**

*   It's a cloud-based (SaaS) software solution designed for businesses that sell to other businesses (B2B).
*   It helps mid-market companies (typically $50M-$1B revenue, 100-1000+ employees) manage customer relationships throughout the entire customer lifecycle.

**Key Features:**

*   Contact, Lead, and Opportunity Management
*   Sales & Workflow Automation
*   Account Management
*   Reporting and Analytics
*   Email Marketing Integration
*   Integration with other business systems
*   Mobile Access & Customization
*   Security & Customer Support

**Why It's Important for Mid-Market Companies:**

*   Scalability & Efficiency
*   Improved Sales Performance & Customer Relationships
*   Data-Driven Decision Making
*   Competitive Advantage
*   Collaboration & Centralized Data

**Choosing the Right CRM:**

*   Consider specific business needs, integration cap